### Install azureml-pipeline for pipeline SDK

In [1]:
!python -m pip install azureml-pipeline==1.0.81

### Install azureml-designer-tools for register SDK

In [2]:
!python -m pip install keyring artifacts-keyring
!python -m pip install azureml-designer-tools==0.1.10.post8799295 --extra-index-url=https://msdata.pkgs.visualstudio.com/_packaging/azureml-modules%40Local/pypi/simple/

In [12]:
from new_module_sdk import Workspace, DatasetX, ModuleStepX, ExperimentX
from azureml.studio.tools.module_repo import ModuleRepo

### Get workspace

In [4]:
workspace_name = ""
resource_group_name = ""
subscription_id = ""

if workspace_name:
    ws = workspace.get(name=workspace_name, resource_group=resource_group_name, subscription_id=subscription_id)
else:
    ws = Workspace.from_config()

### Register module

In [5]:
ModuleRepo.register(workspace=ws, module_source="https://github.com/hirofumi-s-friends/recommenders/tree/cat_demo")

### Get dataset

In [6]:
dataset = DatasetX.get_by_data_reference(ws, "GenericCSV/Movie_Ratings")

### Get modules

In [7]:
stratified_splitter = ModuleStepX.get(ws, name='Stratified Splitter')
sar_train = ModuleStepX.get(ws, name='SAR Train')
sar_score = ModuleStepX.get(ws, name='SAR Score')

Initializing module: Stratified Splitter
Get inputs
inputs.input_path

Get outputs
outputs.output_train_data
outputs.output_test_data

Get params
params.ratio = None, is_optional=False
params.user_column = None, is_optional=False
params.item_column = None, is_optional=False
params.seed = '42', is_optional=False

Initializing module: SAR Train
Get inputs
inputs.input_path

Get outputs
outputs.output_model

Get params
params.user_column = None, is_optional=False
params.item_column = None, is_optional=False
params.rating_column = None, is_optional=False
params.timestamp_column = None, is_optional=False
params.normalize = 'False', is_optional=False
params.time_decay = 'False', is_optional=False

Initializing module: SAR Score
Get inputs
inputs.trained_model
inputs.dataset_to_score

Get outputs
outputs.score_result

Get params
params.score_type = 'Item recommendation', is_optional=False
params.items_to_predict = None, is_optional=True
params.ranking_metric = None, is_optional=True
params.re

### Set inputs, outputs and parameters

In [8]:
stratified_splitter.inputs.input_path = dataset
stratified_splitter.params.user_column = 'UserId'
stratified_splitter.params.item_column = 'MovieId'
stratified_splitter.params.seed = 42
stratified_splitter.params.ratio = 0.8

In [9]:
sar_train.inputs.input_path = stratified_splitter.outputs.output_train_data
sar_train.params.user_column = 'UserId'
sar_train.params.item_column = 'MovieId'
sar_train.params.rating_column = 'Rating'
sar_train.params.timestamp_column = 'Timestamp'

In [10]:
sar_score.inputs.trained_model = sar_train.outputs.output_model
sar_score.inputs.dataset_to_score = stratified_splitter.outputs.output_test_data
sar_score.params.score_type = 'Rating prediction'
sar_score.params.items_to_predict = 'Items in score set'

### Submit pipeline

In [11]:
steps = [stratified_splitter, sar_train, sar_score]
ExperimentX.submit(ws, steps, 'test_experiment_sar')

ModuleStep Stratified Splitter
Inputs:  {'Input_path': $AZUREML_DATAREFERENCE_Dataset}
Outputs:  {'Output_train_data': $AZUREML_DATAREFERENCE_9f62106350254b70ad1cf604c0a72a93, 'Output_test_data': $AZUREML_DATAREFERENCE_6d40d86f0c1c4b609529a13c040be407}
Parameters:  {'Ratio': 0.8, 'User column': 'UserId', 'Item column': 'MovieId', 'Seed': 42}


ModuleStep SAR Train
Inputs:  {'Input_path': $AZUREML_DATAREFERENCE_9f62106350254b70ad1cf604c0a72a93}
Outputs:  {'Output_model': $AZUREML_DATAREFERENCE_f650fbf266d84f85ba572c9860015099}
Parameters:  {'User column': 'UserId', 'Item column': 'MovieId', 'Rating column': 'Rating', 'Timestamp column': 'Timestamp', 'Normalize': "'False'", 'Time decay': "'False'"}


ModuleStep SAR Score
Inputs:  {'Trained_model': $AZUREML_DATAREFERENCE_f650fbf266d84f85ba572c9860015099, 'Dataset_to_score': $AZUREML_DATAREFERENCE_6d40d86f0c1c4b609529a13c040be407}
Outputs:  {'Score_result': $AZUREML_DATAREFERENCE_7191b6f714664e9bbd6c50501561b5b7}
Parameters:  {'Score type'